In [0]:
import numpy as np
import torch
import pickle
import torch.utils.data as data_utils
import pandas as pd
import torch.nn as nn
import time
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('/content/gdrive/My Drive/features_50.pkl', 'rb') as f:
  features = pickle.load(f)
  
with open('/content/gdrive/My Drive/folds_50.pkl', 'rb') as f:
  folds = pickle.load(f)
  
with open('/content/gdrive/My Drive/labels_50.pkl', 'rb') as f:
  labels = pickle.load(f)

## Numpy to Tensors

In [8]:
print(len(features), len(folds), len(labels))

3676 3676 3676


In [9]:
features[0].shape

(50, 50, 3)

In [0]:
features_arr = np.asarray(features, dtype=np.float32)
labels_arr = np.asarray(labels, dtype=np.int64)
folds_array = np.asarray(folds)

In [0]:
features_arr = np.swapaxes(features_arr, 1, 3)
features_arr = np.swapaxes(features_arr, 2, 3)

In [12]:
print(features_arr.shape, labels_arr.shape)

(3676, 3, 50, 50) (3676,)


In [13]:
unique_items, counts = np.unique(folds, return_counts=True)
unique_items, counts

(array(['fold1', 'fold2', 'fold3', 'fold4'], dtype='<U5'),
 array([873, 888, 925, 990]))

In [0]:
sum1 = counts[0]
sum12 = counts[0] + counts[1]
sum23 = sum12 + counts[2]
sum34 = sum23 + counts[3]

In [0]:
y_te1 = labels_arr[0:sum1]
y_tr1 = labels_arr[sum1:]
X_te1 = features_arr[0:sum1, :, :, :]
X_tr1 = features_arr[sum1:, :, :, :]
folds_1 = np.delete(folds_array, list(range(0, sum1)))

In [0]:
y_te2 = labels_arr[sum1:sum12]
y_tr2 = np.concatenate([labels_arr[0:sum1], labels_arr[sum12:]])
X_te2 = features_arr[sum1:sum12, :, :, :]
X_tr2 = np.delete(features_arr, slice(sum1, sum12), axis=0)
folds_2 = np.delete(folds_array, list(range(sum1, sum12)))

In [0]:
y_te3 = labels_arr[sum12:sum23]
y_tr3 = np.concatenate([labels_arr[0:sum12], labels_arr[sum23:]])
X_te3 = features_arr[sum12:sum23, :, :, :]
X_tr3 = np.delete(features_arr, slice(sum12,sum23), axis=0)
folds_3 = np.delete(folds_array, list(range(sum12,sum23)))

In [0]:
y_te4 = labels_arr[sum23:sum34]
y_tr4 = np.concatenate([labels_arr[0:sum23], labels_arr[sum34:]])
X_te4 = features_arr[sum23:sum34, :, :, :]
X_tr4 = np.delete(features_arr, slice(sum23,sum34), axis=0)
folds_4 = np.delete(folds_array, list(range(sum23,sum34)))

In [0]:
group_list = [folds_1, folds_2, folds_3, folds_4]
X_train_list = [X_tr1, X_tr2, X_tr3, X_tr4]
y_train_list = [y_tr1, y_tr2, y_tr3, y_tr4]
X_test_list = [X_te1, X_te2, X_te3, X_te4]
y_test_list = [y_te1, y_te2, y_te3, y_te4]

## Model


In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np


train_transformations = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transformations = transforms.Compose([
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))

])

class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

train_loaders = []
test_loaders = []
for i in range(4):
  
  numpy_data_tr = X_train_list[i]
  numpy_target_tr = y_train_list[i]
  
  
  dataset_tr = MyDataset(numpy_data_tr, numpy_target_tr, transform=train_transformations)
  loader_tr = DataLoader(
      dataset_tr,
      batch_size=32,
      shuffle=False,
      num_workers=2,
      pin_memory=torch.cuda.is_available()
  )
  
  numpy_data_te = X_test_list[i]
  numpy_target_te = y_test_list[i]
  
  dataset_te = MyDataset(numpy_data_te, numpy_target_te, transform=test_transformations)
  loader_te = DataLoader(
      dataset_te,
      batch_size=32,
      shuffle=False,
      num_workers=2,
      pin_memory=torch.cuda.is_available()
  )
  
  train_loaders.append(loader_tr)
  test_loaders.append(loader_te)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [158]:
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import f1_score

class SimpleNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride = 2)
        self.dropout1 = nn.Dropout(.03)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv22 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv23 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv24 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=12)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride = 2)
        self.dropout2 = nn.Dropout(.03)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=36, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.conv42 = nn.Conv2d(in_channels=36, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.conv43 = nn.Conv2d(in_channels=36, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.conv44 = nn.Conv2d(in_channels=36, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=36)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride = 2)
        self.dropout3 = nn.Dropout(.03)
        self.conv5 = nn.Conv2d(in_channels=36, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.conv61 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.conv62 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.conv63 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=48)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride = 2)
        self.avgpool = nn.AvgPool2d(kernel_size=4)           
        self.fc = nn.Linear(in_features=432, out_features=num_classes)

    def forward(self, input):
#         print(input.shape)

        output = self.conv1(input)
        output = self.bn1(output)

        output = self.relu1(output)
    
#         print(output.shape)

        output = self.pool1(output)
        output = self.dropout1(output)

#         print(output.shape)

#         print(output.shape)
#         print(output.shape)
        
        output = self.conv2(output)
        output = self.conv22(output)
        output = self.conv23(output)
        output = self.conv24(output)
        output = self.bn2(output)

        output = self.relu2(output)

#         print(output.shape)
#         print(output.shape)
#         print(output.shape)

        output = self.pool2(output)
        output = self.dropout2(output)

#         print(output.shape)


        output = self.conv3(output)
        output = self.conv4(output)
        output = self.conv42(output)
        output = self.conv43(output)
        output = self.conv44(output)
        output = self.bn3(output)

        output = self.relu3(output)

#         print(output.shape)
#         print(output.shape)

#         print(output.shape)
        output = self.pool3(output)
        output = self.dropout3(output)
  #         output = self.pool3(output)

        output = self.conv5(output)
        output = self.conv6(output)
        output = self.conv61(output)
        output = self.conv62(output)
        output = self.conv63(output)
        output = self.bn4(output)
        output = self.relu4(output)

        output = self.pool4(output)
#         output = self.dropout4(output)

        outout = self.avgpool(output)

#         print(output.shape)
        output = output.view(-1, 432)
#         print(output.shape)
        output = self.fc(output)
#         print(output.shape)
        return output



log_freq = 40

cuda_avail = torch.cuda.is_available()

model = SimpleNet()

if cuda_avail:
    model.cuda()

optimizer = Adam(model.parameters(), lr=0.00008)
loss_fn = nn.CrossEntropyLoss()



def save_models(epoch):
    torch.save(model.state_dict(), "conv_{}.model".format(epoch))
    print("Checkpoint saved")

def test():
    model.eval()
    test_acc = 0.0
    running_loss = 0.0
    running_f1 = 0.0
    for i, (images, labels) in enumerate(test_loaders[1]):
      
        if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

        output = model(images)

        loss = loss_fn(output, labels)
        running_loss += loss.item()
        y_pred = output.argmax(dim=-1)  
        
        running_f1 += f1_score(labels.cpu(), y_pred.cpu(), average='macro')
        
    average_loss = running_loss / len(test_loaders[1])
    average_f1 = running_f1 / (len(test_loaders[1]))
    
    
    print(f'Val Loss: {average_loss:.5f} Val f1: {average_f1:.5f}\n')
    test_acc = test_acc / len(test_loaders[1])
    return average_f1

  
  
def train(num_epochs):
    best_f1 = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_f1 = 0.0
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loaders[1]):
            
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            output = model(images)
            loss = loss_fn(output,labels)
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
           
            running_loss += loss.item()
            y_pred = output.argmax(dim=-1)
            running_f1 += f1_score(labels.cpu(), y_pred.cpu(), average='macro')
            if i % log_freq == log_freq - 1:
              average_loss = running_loss / len(train_loaders[1])
              average_f1 = running_f1 / len(train_loaders[1])
              print(f'Mini-batch: {i + 1}/{len(train_loaders[1])} '
                  f'Loss: {average_loss:.5f} f1: {average_f1:.5f}')
              running_loss = 0.0
              running_f1 = 0.0

        average_f1 = test()
        
        if average_f1 > best_f1:
            save_models(epoch)
            best_f1 = average_f1

if __name__ == "__main__":
    train(25)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Mini-batch: 40/88 Loss: 0.98119 f1: 0.06156
Mini-batch: 80/88 Loss: 0.92519 f1: 0.06199
Val Loss: 2.06181 Val f1: 0.09851

Checkpoint saved
Mini-batch: 40/88 Loss: 0.87391 f1: 0.10455
Mini-batch: 80/88 Loss: 0.84405 f1: 0.11486
Val Loss: 1.94572 Val f1: 0.20077

Checkpoint saved
Mini-batch: 40/88 Loss: 0.80134 f1: 0.14909
Mini-batch: 80/88 Loss: 0.78804 f1: 0.14328
Val Loss: 1.88711 Val f1: 0.22956

Checkpoint saved
Mini-batch: 40/88 Loss: 0.74897 f1: 0.17344
Mini-batch: 80/88 Loss: 0.75200 f1: 0.14890
Val Loss: 1.84751 Val f1: 0.24192

Checkpoint saved
Mini-batch: 40/88 Loss: 0.70633 f1: 0.18654
Mini-batch: 80/88 Loss: 0.71760 f1: 0.16471
Val Loss: 1.78455 Val f1: 0.27529

Checkpoint saved
Mini-batch: 40/88 Loss: 0.67387 f1: 0.20003
Mini-batch: 80/88 Loss: 0.68460 f1: 0.17800
Val Loss: 1.77506 Val f1: 0.28900

Checkpoint saved
Mini-batch: 40/88 Loss: 0.64169 f1: 0.21224
Mini-batch: 80/88 Loss: 0.65872 f1: 0.19243
Val Loss: 1.74905 Val f1: 0.31728

Checkpoint saved
Mini-batch: 40/88 Lo